## Omer contour from rembg needs testing

In [1]:
# %pip install rembg
# %pip install onnxruntime

In [2]:
class AttrDict(dict):
    def __init__(self, *args, **kwargs):
        super(AttrDict, self).__init__(*args, **kwargs)
        self.__dict__ = self

In [3]:
"""
Tests for the plugin system
"""
import pytest
import os
import sys
import tempfile
from pathlib import Path
import shutil
import importlib
import time


# Import plugin modules
from pyptv.plugins.ext_sequence_rembg_contour import Sequence
from pyptv.ptv import py_start_proc_c, py_trackcorr_init, py_sequence_loop


In [4]:
    
exp_path = Path('/home/user/Downloads/exp2').resolve()
first = 10000
last = 10004

start = time.time()

try:
    exp_path = Path(exp_path).resolve()
    print(f"Inside main of pyptv_batch, exp_path is {exp_path} \n")
    os.chdir(exp_path)
    
    print(f"double checking that its inside {Path.cwd()} \n")
except Exception:
    raise ValueError(f"Wrong experimental directory {exp_path}")

# RON - make a res dir if it not found

res_path = exp_path / "res"

if not res_path.is_dir():
    print(" 'res' folder not found. creating one")
    res_path.mkdir(parents=True, exist_ok=True)

seq_first = int(first)
seq_last = int(last)

print((seq_first, seq_last))

# read the number of cameras
with open("parameters/ptv.par", "r") as f:
    n_cams = int(f.readline())

cpar, spar, vpar, track_par, tpar, cals, epar = py_start_proc_c(n_cams=n_cams)

spar.set_first(seq_first)
spar.set_last(seq_last)

exp = {
'cpar':cpar,
'spar':spar,
'vpar':vpar,
'track_par':track_par,
'tpar':tpar,
'cals':cals,
'epar':epar,
'n_cams':n_cams,
    }


# use dataclass to convert dictionary keys to attributes
exp = AttrDict(exp)

#    py_sequence_loop(exp)
from pyptv.ptv import run_plugin
plugin_dir = Path('/home/user/Documents/repos/pyptv/pyptv') / 'plugins'
sys.path.append(str(plugin_dir))

print(f"Plugin directory contents: {list(plugin_dir.glob('*.py'))}")

plugin_file =  plugin_dir / 'ext_sequence_rembg_contour.py'


if not plugin_file.exists():
    raise FileNotFoundError(f"Plugin file not found at {plugin_file}")

print(f"Plugin directory contents: {list(plugin_dir.glob('*.py'))}")

plugin = importlib.import_module('ext_sequence_rembg_contour')


sequence = plugin.Sequence(exp=exp)
sequence.do_sequence()


tracker = py_trackcorr_init(exp)
tracker.full_forward()

end = time.time()
print("time lapsed %f sec" % (end - start))



Inside main of pyptv_batch, exp_path is /home/user/Downloads/exp2 

double checking that its inside /home/user/Downloads/exp2 

(10000, 10004)
Plugin directory contents: [PosixPath('/home/user/Documents/repos/pyptv/pyptv/plugins/ext_sequence_contour.py'), PosixPath('/home/user/Documents/repos/pyptv/pyptv/plugins/ext_sequence_denis.py'), PosixPath('/home/user/Documents/repos/pyptv/pyptv/plugins/ext_tracker_denis.py'), PosixPath('/home/user/Documents/repos/pyptv/pyptv/plugins/ext_sequence_rembg_contour.py'), PosixPath('/home/user/Documents/repos/pyptv/pyptv/plugins/ext_sequence_rembg.py')]
Plugin directory contents: [PosixPath('/home/user/Documents/repos/pyptv/pyptv/plugins/ext_sequence_contour.py'), PosixPath('/home/user/Documents/repos/pyptv/pyptv/plugins/ext_sequence_denis.py'), PosixPath('/home/user/Documents/repos/pyptv/pyptv/plugins/ext_tracker_denis.py'), PosixPath('/home/user/Documents/repos/pyptv/pyptv/plugins/ext_sequence_rembg_contour.py'), PosixPath('/home/user/Documents/repo

FileNotFoundError: No such file: '/media/user/ExtremePro/omer/mask_centers_sam2/cam0_mask_centers/frame_10000_centers.tiff'